In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score


import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score as sk_roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [3]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip')#.iloc[:10000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip')#.iloc[:10000,:]

In [4]:
df_oof_train = pd.DataFrame()
df_oof_test = pd.DataFrame()

In [5]:
df_oof_vae_train = pd.read_pickle(f'{folder_path}/oof_vae_train', compression='gzip')
df_oof_vae_test = pd.read_pickle(f'{folder_path}/oof_vae_test', compression='gzip')
df_oof_train = df_oof_vae_train
df_oof_test = df_oof_vae_test

In [6]:
df_oof_lgbm_train = pd.read_pickle(f'{folder_path}/oof_lgbm_train', compression='gzip')
df_oof_lgbm_test = pd.read_pickle(f'{folder_path}/oof_lgbm_test', compression='gzip')
df_oof_lgbm_test['predict'] = df_oof_lgbm_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_lgbm_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_lgbm'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_lgbm_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_lgbm'}), how='left', on='TransactionID')

In [7]:
df_oof_xgbm_train = pd.read_pickle(f'{folder_path}/oof_xgbm_train', compression='gzip')
df_oof_xgbm_test = pd.read_pickle(f'{folder_path}/oof_xgbm_test', compression='gzip')
df_oof_xgbm_test['predict'] = df_oof_xgbm_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_xgbm_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_xgbm'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_xgbm_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_xgbm'}), how='left', on='TransactionID')

In [8]:
df_oof_mlp_train = pd.read_pickle(f'{folder_path}/oof_mlp_train', compression='gzip')
df_oof_mlp_test = pd.read_pickle(f'{folder_path}/oof_mlp_test', compression='gzip')
df_oof_mlp_test['predict'] = df_oof_mlp_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_mlp_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_mlp'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_mlp_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_mlp'}), how='left', on='TransactionID')

In [9]:
df_oof_train.head()

,loss,kle,reconstructione,mse,TransactionID,oof_lgbm,oof_xgbm,oof_mlp
0,87.781677,-0.000000,87.781677,0.237053,2987000,0.005126,0.007788,0.015279
1,75.931198,0.007184,75.924011,0.180677,2987001,0.002398,0.010556,0.034451
2,79.129272,-0.000000,79.129272,0.156918,2987002,0.001919,0.012954,0.002952
3,83.263702,0.000120,83.263580,0.191927,2987003,0.000922,0.002685,0.000013
4,78.967186,0.007210,78.959976,0.201674,2987004,0.001328,0.001727,0.012591


In [10]:
df_oof_test.head()

,TransactionID,loss,kle,reconstructione,mse,oof_lgbm,oof_xgbm,oof_mlp
0,3663549,76.213234,0.131185,76.082048,0.170198,0.000186,0.000334,0.000497
1,3663550,83.605424,0.125992,83.479432,0.199555,0.001328,0.002672,0.000377
2,3663551,80.684792,0.125003,80.559789,0.155993,0.001123,0.000482,0.001226
3,3663552,80.136013,0.127257,80.008757,0.156172,0.000416,0.001683,0.004226
4,3663553,78.773938,0.126601,78.647337,0.149714,0.000441,0.000414,0.011174


In [11]:
df_train = pd.merge(df_train, df_oof_train, how='left', on='TransactionID')
df_test = pd.merge(df_test, df_oof_test, how='left', on='TransactionID')

In [12]:
params = {
    'n_estimators':500,
    'max_depth':9,
    'learning_rate':0.05,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'missing':-999,
    'random_state':2019,
    'tree_method':'gpu_hist'  # THE MAGICAL PARAMETER
}

In [13]:
def process(params, folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
        
        clf = xgb.XGBClassifier(**params)
        clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values)
        
        y_trn_pred = clf.predict_proba(X.iloc[trn_idx].values)[:,1]        
        y_val_pred = clf.predict_proba(X.iloc[test_idx].values)[:,1]
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict_proba(X_test.values)[:,1]
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        if is_output_feature_importance:
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [15]:
select_columns = ['PCA_C_1',
 'C13',
 'C14',
 'C1_fq_enc',
 'PCA_C_2',
 'V70',
 'D2',
 'M4_target_mean',
 'V317',
 'uid3_TransactionAmt_mean',
 'card3_TransactionAmt_mean',
 'R_emaildomain',
 'C11',
 'D15_to_mean_addr1',
 'V258',
 'uid3_TransactionAmt_std',
 'card2_fq_enc',
 'card2_TransactionAmt_mean',
 'addr1__card1',
 'TransactionAmt',
 'ProductCD_target_mean',
 'uid3_fq_enc',
 'M6',
 'card5__P_emaildomain',
 'C5',
 'C2',
 'D3',
 'P_emaildomain',
 'V283',
 'card2',
 'V45',
 'uid_TransactionAmt_std',
 'TransactionAmt_decimal',
 'V189',
 'card1_fq_enc',
 'card1_count_full',
 'card3',
 'card6_count_full',
 'V281',
 'device_version',
 'V49',
 'V219',
 'V160',
 'DeviceInfo_fq_enc',
 'V224',
 'V220',
 'R_emaildomain_fq_enc',
 'id_02_to_std_card4']

In [17]:
columns = select_columns + df_oof_train.columns.drop(['TransactionID']).tolist()

In [19]:
folds = TimeSeriesSplit(n_splits=5)

In [20]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(params, folds, df_train,df_test, columns, is_output_feature_importance=0, verbose=1)

Fold 1 finished in 0:01:00.936813
Fold 2 finished in 0:01:58.579901
Fold 3 finished in 0:03:02.897928
Fold 4 finished in 0:03:59.095650
Fold 5 finished in 0:04:43.979245
------------------------------
Training has finished.
Total training time is 0:14:51.371414
Mean AUC: 0.9363151044367365 0.9991845473002547
------------------------------


In [21]:
his

,val_auc,trn_auc,y_val_pred,y_test_pred,test_idx
0,0.928564,1.000000,"[0.0026149782, 0.0005775193, 0.00017311459, 0....","[8.4444015e-05, 0.00046996126, 9.914195e-05, 0...","[98425, 98426, 98427, 98428, 98429, 98430, 984..."
1,0.945101,0.999936,"[0.37206224, 0.00019528682, 0.059499238, 0.000...","[3.3701213e-05, 0.00056721596, 0.00011139754, ...","[196848, 196849, 196850, 196851, 196852, 19685..."
2,0.929942,0.999500,"[0.00013347359, 0.0031568112, 0.00869652, 0.00...","[4.1247862e-05, 0.00018055714, 7.342059e-05, 0...","[295271, 295272, 295273, 295274, 295275, 29527..."
3,0.945533,0.998642,"[0.0008261419, 0.0029900002, 0.025698349, 5.92...","[8.32846e-05, 0.0001674013, 0.0001923025, 0.00...","[393694, 393695, 393696, 393697, 393698, 39369..."
4,0.932435,0.997844,"[0.011265962, 0.0007474771, 0.0115833515, 0.00...","[3.349653e-05, 0.00011908375, 0.00022510684, 0...","[492117, 492118, 492119, 492120, 492121, 49212..."


In [22]:
df_test_pred['isFraud']=df_test_pred[3]
df_test_pred[['TransactionID','isFraud']].to_pickle(f'{folder_path}/oof_stacking_xgbm_tss5', compression='gzip')

In [24]:
folds = KFold(n_splits=8, shuffle=False, random_state=42)

In [25]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(params, folds, df_train,df_test, select_columns, is_output_feature_importance=0, verbose=1)

Fold 1 finished in 0:05:31.194081
Fold 2 finished in 0:06:07.449240
Fold 3 finished in 0:05:23.651994
Fold 4 finished in 0:05:57.962866
Fold 5 finished in 0:05:29.616574
Fold 6 finished in 0:05:32.574375
Fold 7 finished in 0:05:29.911786
Fold 8 finished in 0:05:33.239358
------------------------------
Training has finished.
Total training time is 0:45:09.703493
Mean AUC: 0.9387051127224363 0.9939887981753599
------------------------------


In [26]:
his

,val_auc,trn_auc,y_val_pred,y_test_pred,test_idx
0,0.922666,0.993891,"[0.006871131, 0.012053348, 0.015563545, 0.0014...","[0.00027233112, 0.0018955157, 0.0004632995, 0....","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,0.935171,0.994230,"[0.0040979553, 0.00046151105, 0.0041206055, 0....","[0.0002571506, 0.002537921, 0.00040768503, 0.0...","[73818, 73819, 73820, 73821, 73822, 73823, 738..."
2,0.939759,0.993967,"[0.00080712396, 0.0006266288, 0.22959797, 0.00...","[0.00015871323, 0.0020603696, 0.00031359761, 0...","[147636, 147637, 147638, 147639, 147640, 14764..."
3,0.947926,0.993782,"[0.00291602, 0.0022610142, 0.009222973, 0.1006...","[0.0001456559, 0.0021266616, 0.0008335324, 0.0...","[221454, 221455, 221456, 221457, 221458, 22145..."
4,0.931478,0.994175,"[0.010124747, 0.013324458, 0.00018594188, 0.00...","[0.00036074064, 0.0022272703, 0.0003972786, 0....","[295272, 295273, 295274, 295275, 295276, 29527..."
5,0.952062,0.994030,"[0.056929585, 0.0054036444, 0.0012144389, 0.00...","[0.0003165709, 0.0026185226, 0.00059088756, 0....","[369089, 369090, 369091, 369092, 369093, 36909..."
6,0.947273,0.993745,"[0.0003355001, 0.0005538683, 0.004608299, 0.01...","[0.00038355496, 0.0040509007, 0.00050814875, 0...","[442906, 442907, 442908, 442909, 442910, 44291..."
7,0.933306,0.994090,"[0.0033968918, 0.00016805943, 0.022597276, 0.1...","[0.00045174191, 0.0016257539, 0.00055260985, 0...","[516723, 516724, 516725, 516726, 516727, 51672..."


In [27]:
df_test_pred['isFraud']=df_test_pred[np.arange(8)].mean(axis=1)

In [28]:
df_test_pred[['TransactionID','isFraud']].to_pickle(f'{folder_path}/oof_stacking_xgbm_kf8', compression='gzip')